In [1]:
import pandas as pd
df = pd.read_csv('./data/rg65_spanish.txt', delimiter =  "\t", header = None)
df.columns = ['word1', 'word2', 'org_sim']

In [2]:
# df['org_sim'] = df['org_sim'] / 4

In [3]:
# mm_model_path = 'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
# bm_model_path = 'benchmark_model_200000_1.0_15v7'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [4]:
mm_model_path = "finetuned_multimodal_lstm"#'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
bm_model_path = 'finetuned_benchmark_lstm'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else "gpu"
from multimodal_model import BenchmarkLSTM, MMLSTM
from multi_bpe import MultiBPE
multi_bpe = MultiBPE()
def load_model(loaded_model, is_benchmark, is_multimodal, train_visual_module):
    if is_benchmark:
        mm_model = BenchmarkLSTM().to(device)
    else:
        mm_model = MMLSTM(is_multimodal=is_multimodal,
                     train_visual_module=train_visual_module).to(device)
    
    mm_model.load_state_dict(torch.load(f'./saved_models/{loaded_model}'))
    mm_model.eval()
    return mm_model

mm_model = load_model(mm_model_path, False, True, False)
bm_model = load_model(bm_model_path, True, False, False)

In [6]:
def cosine_score(w1, w2, model, name='emb_layer.weight'):
    tcs = torch.nn.CosineSimilarity(dim=1)
    e1 = multi_bpe.encode(w1, False, False, None)
    e2 = multi_bpe.encode(w2, False, False, None)
    if len(e1) > 1 or len(e2) > 1:
        a = model.state_dict()[name][e1[0]]
        for idx in e1[1:]:
            a = torch.concat((a, model.state_dict()[name][idx]))
        b = model.state_dict()[name][e2[0]]
        for idx in e2[1:]:
            b = torch.concat((b, model.state_dict()[name][idx]))
        return 999
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
    else:
        a = model.state_dict()[name][e1[0]]
        b = model.state_dict()[name][e2[0]]
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()

def cosine_sim_model(df, model, name):
    l = []    
    for i in range(len(df)):
        w1, w2 = df.word1[i], df.word2[i]
        l.append(cosine_score(w1, w2, model, name))
    return l

In [7]:
df['bm_sim'] = cosine_sim_model(df, bm_model, 'fc.weight')
df['mm_sim'] = cosine_sim_model(df, mm_model, 'fc.weight')

<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()


In [8]:
df = df[df.bm_sim != 999].reset_index(drop=True)
df

,word1,word2,org_sim,bm_sim,mm_sim
0,gallo,viaje,0.04,0.826047,0.838862
1,fruta,caldera,0.21,0.877928,0.897251
2,chico,gallo,0.29,0.866872,0.819806
3,vaso,mago,0.08,0.869517,0.861246
4,costa,bosque,1.17,0.898962,0.914182
5,chico,sabio,0.79,0.862871,0.817291
6,selva,cementerio,0.38,0.882334,0.918193
7,comida,gallo,1.54,0.775237,0.823091
8,cementerio,bosque,0.79,0.912350,0.909964
9,costa,colina,1.50,0.914837,0.916587


In [9]:
df.corr(method='pearson')

,org_sim,bm_sim,mm_sim
org_sim,1.000000,0.558360,0.603707
bm_sim,0.558360,1.000000,0.860628
mm_sim,0.603707,0.860628,1.000000


In [10]:
import pingouin as pg

#find partial correlation between hours and exam score while controlling for grade
pg.partial_corr(data=df, x='mm_sim', y='org_sim', covar='bm_sim')

,n,r,CI95%,p-val
pearson,18,0.291545,"[-0.22, 0.68]",0.25621


In [11]:
00.182981**2

0.033482046361

In [12]:
0.375926**2

0.141320357476

In [13]:
0.554627**2

0.30761110912899997